In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-h65k69np
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-h65k69np
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=ec4a88cb45eb50f4ffa435796c3708861cf25f3641d0ae715f2dd067eacd1d24
  Stored in directory: /tmp/pip-ephem-wheel-cache-6t2kxokl/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%reload_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [ ]:
%%cu 
#include <stdio.h>
#include <stdlib.h>

#define N  2

__global__ void MatAdd(int A[][N] ,int B[][N] ,int C[][N]){
    int i = threadIdx.x;
    int j = threadIdx.y;
    C[i][j] = A[i][j]+B[i][j];
}

int main(){
    int A[N][N]={{1,2},{3,4}};
    int B[N][N]={{5,6},{7,8}};
    int C[N][N]={{0,0},{0,0}};

    int (*pA)[N] ,(*pB)[N] , (*pC)[N];

    cudaMalloc((void**)&pA,(N*N)*sizeof(int));
    cudaMalloc((void**)&pB,(N*N)*sizeof(int));
    cudaMalloc((void**)&pC,(N*N)*sizeof(int));

    cudaMemcpy(pA,A,(N*N)*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(pB,B,(N*N)*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(pC,C,(N*N)*sizeof(int),cudaMemcpyHostToDevice);
    int numBlocks=1;
    dim3 threadsPerBlock(N,N);
    MatAdd<<<numBlocks,threadsPerBlock>>>(pA,pB,pC);
    cudaMemcpy(C,pC,(N*N)*sizeof(int),cudaMemcpyDeviceToHost);
    printf("A = \n");
    for(int i=0;i<N;i++){
        for(int j=0;j<N;j++){
            printf("%d, ",A[i][j]);
        }printf("\n");
    }
    printf("B = \n");
    for(int i=0;i<N;i++){
        for(int j=0;j<N;j++){
            printf("%d, ",B[i][j]);
        }printf("\n");
    }
    
    printf("C = \n");
    for(int i=0;i<N;i++){
        for(int j=0;j<N;j++){
            printf("%d, ",C[i][j]);
        }printf("\n");
    }

    cudaFree(pA);
    cudaFree(pB);
    cudaFree(pC);
    return 0;

}

A = 
1, 2, 
3, 4, 
B = 
5, 6, 
7, 8, 
C = 
6, 8, 
10, 12, 

